<font size=50 color=darkblue>Shortest Path ILP</font>

# Problem modelling in LP format

## Import necessary modules
- Import function `read_model` from DOcplex to read the Shortest-Path ILP from a temporary file
- Module `tempfile` is imported to create the temporary file
- Import function `remove` to delete the temporary file after reading it (required for python version < 3.12)

In [ ]:
from docplex.mp.model_reader import read_model
import tempfile
from os import remove

## Shortest-Path ILP <font size=3>(variables are colored blue)</font>
**Minimize**
### $$\sum_{(u,v)\in \mathcal{E}}c_{(u,v)}\cdot \color{blue}x_{(u,v)}$$
**Subject to**
### \begin{align*}
{\color{blue}x}_{({\color{red}t},{\color{green}s})} = 1\,\qquad&\\
\sum_{(u,v)\in{\color{red}\delta}^{\color{red}-}_{(v)}}{\color{blue}x_{(u,v)}} - \sum_{(v,u)\in{\color{green}\delta}^{\color{green}+}_{(v)}}{\color{blue}x_{(v,u)}} = 0,\qquad&\forall v\in \mathcal{V}\\
{\color{blue}x_{(u,v)}}\in\left\{0,1\right\},\qquad&\forall(u,v)\in \mathcal{E}
\end{align*}

## Display the network
<img src='img/sp.png' width=700/>

## Write the Shortest-Path ILP model (in LP format), which is assigned to the variable `sp_str`
### Note
- Binary variables are declared in the `Binaries` section. No bound declarations for binary variables are necessary.

In [ ]:
sp_str = '''
Minimize
 obj: 15 x(0,1) + 20 x(0,2) + 10 x(1,3) + 25 x(1,4) + 15 x(2,3) + 20 x(2,5)
      + 20 x(3,4) + 15 x(3,5) + 30 x(3,6) + 10 x(4,6) + 20 x(5,6)
      
Subject To
 demand_satisfaction: x(6,0) = 1
 flow_conservation(0): x(6,0) - x(0,1) - x(0,2) = 0
 flow_conservation(1): x(0,1) - x(1,3) - x(1,4) = 0
 flow_conservation(2): x(0,2) - x(2,3) - x(2,5) = 0
 flow_conservation(3): x(1,3) + x(2,3) - x(3,4) - x(3,5) - x(3,6) = 0
 flow_conservation(4): x(1,4) + x(3,4) - x(4,6) = 0
 flow_conservation(5): x(2,5) + x(3,5) - x(5,6) = 0
 flow_conservation(6): x(3,6) + x(4,6) + x(5,6) - x(6,0) = 0

Binaries
 x(0,1) x(0,2) x(1,3) x(1,4) x(2,3) x(2,5) x(3,4) x(3,5) x(3,6) x(4,6) x(5,6) x(6,0)

End
'''

## To import the ILP model to DOcplex
- Store the model to a temporary file
- Have DOcplex read the model from the temporary file

In [ ]:
# Store string to tmp file
with tempfile.TemporaryFile(delete=False) as tmp:
    tmp.write(sp_str.encode('utf-8'))
    tmp.close()
    # Have DOcplex read the string
    sp_ILP = read_model(filename=tmp.name, model_name='Shortest Path')
    # Delete the temporary file
    remove(tmp.name)

## Summarize the model

In [ ]:
sp_ILP.print_information()

## Solve the ILP and display the result

In [ ]:
sp_sol = sp_ILP.solve()
if sp_sol:
    sp_sol.display()

# Result Visualization

## Import visualization modules
- `igraph`, `matplotlib`, `re`

In [ ]:
import igraph as ig
import matplotlib.pyplot as plt
import re

## Extract the link list `E`, link costs `c`, source <code><font color='green'>s</font></code>, and sink <code><font color='red'>t</font></code> from the ILP model

In [ ]:
E, c = [], {}

for var in sp_ILP.iter_variables():
    u, v = re.findall('x\((.+),(.+)\)', var.name)[0]
    E.append(e:=(int(u), int(v)))
    c[e] = sp_ILP.objective_expr.get_coef(var)

t,s = re.findall('x\((.+),(.+)\)', str(sp_ILP.get_constraint_by_name('demand_satisfaction').get_left_expr()))[0]
s, t = int(s), int(t)

## Since the link (<font color='red'>t</font>, <font color='green'>s</font>) is imaginary, remove it from the link list `E`

In [ ]:
E.remove((t,s))

## For convenience, extract the solution to a dictionary named `sol_x`

In [ ]:
sol_x = {(u,v): sp_sol.get_value(f'x({u},{v})') for u,v in E}
sol_x

## Instantiate a `Graph` object with module `igraph`
### Notes
- __*Node(s)*__ is/are called __*vertex/vertices*__ in `igraph`
- __*Link(s)*__ is/are called __*edge/edges*__ in `igraph`
- The edge list is sufficient to instantiate a `Graph` object. The vertex list is automatically inferred by `igraph` (based on the tails/heads' indices).

In [ ]:
g = ig.Graph(edges=E, directed=True)

## Visualize the graph

In [ ]:
g.vs['label'] = g.vs.indices
g.vs['size'] = 50
g.vs['color'] = 'white'
g.vs[s,t]['color'] = 'green', 'red'

lnk_flow_max = max(sol_x.values())
g.es['label'] = [c[e] for e in E]
g.es['arrow_size'] = [10 if sol_x[e] > 0 else 6 for e in E]
g.es['arrow_width'] = [10 if sol_x[e] > 0 else None for e in E]
g.es['width'] = [5 if sol_x[e] > 0 else 0.5 for e in E]
g.es['color'] = ['darkblue' if sol_x[e] > 0 else None for e in E]
g.es['label_size'] = 15

fig, ax = plt.subplots()
fig.set_size_inches(10,10)

p = ig.plot(g, layout=g.layout('kk'), edge_label=g.es['label'], edge_background='white', target=ax)
    
plt.show()